# Volvo Trucks Analytics

Imports and global variables go here

In [7]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

#The two truck data set paths are set here.
truck1csv = "../data/trucks/truck1.csv"
truck2csv = "../data/trucks/truck2.csv"

#Number of records desired from the data set.
numberOfRecords = 1000

#If the number of values that are NOT Na type pass this percentage, the row will be deleted
rowNaNThresholdPercent = 75

#The column conversion dictionaries are defined here.
#(!) Consider moving to the utility folder to improve readability. (!)
truck1dict = {
    'Time':'Time (s)',
    '1730_CH9_ AutomaticStartStop':'1730 Automatic Start & Stop (V)',
    '1730_CH10_Truck_Batteries':'1730 Batteries (V)',
    '4649_Ch1_Alternator_250A':'4649 Alternator (A)',
    '4649_Ch2_BattOut_100A':'4649 Battery Out (A)',
    '4649_Ch3_Trailer_50A':'4649 Trailer (A)',
    '4649_Ch4_Inverter_100A':'4649 Inverter (A)',
    '4649_Ch5_Fridge_50A':'4649 Fridge (A)',
    '4649_Ch6_200A_APU_BatteryBank':'4649 Battery Bank (A)',
    '4649_Ch7_200A_BatterySeparator':'4649 Battery Separator (A)',
    '4649_Ch8_APU_BatteryVoltage':'4649 Battery Voltage (V)',
    'ActualEnginePercentTorque':'Engine Output Torque (%)',
    'ActualEnginePercentTorqueHiRes':'HiRes Engine Output Torque (%)',
    'AmbientAirTemperature_V':'Outside Air Temperature (C)',
    'asu_UreaTankTemp':'Urea Tank Temperature (C)',
    'BarometricPressure_BB1_X_E':'Outside Air Pressure (kPa)',
    'BoostPressure':'Boost Pressure (kPa)',
    'CruiseCtrlSetSpeed':'Cruise Control Speed (km/hr)',
    'DrvrDemandEngPercentTorque':'Driver Requested Torque (%)',
    'EngDemandPercentTorque':'Engine Requested Torque (%)',
    'EngDslPrtclateFilterIntakePress':'Exhaust Back Pressure (kPa)',
    'EngExhaustGasTemp':'Temperature of Engine Byproducts (C)',
    'EngineAirIntakeTemp_BB1_X_E':'Temperature of Air Entering Vehicle (C)',
    'EngineCoolantTemp_stat':'Temperature of Engine Liquid (C)',
    'EngineKeyPosition':'Key Position',
    'EngineOilLevel':'Oil Level (%)',
    'EngineOilPressure':'Oil Pressure (kPa)',
    'EngineOilTemperature':'Oil Temperature (C)',
    'EnginePercentLoadAtCurrentSpd':'Engine Load at Current Speed (%)',
    'EngineSpeed':'Engine Speed (rpm)',
    'EngIntakeManifold1Press':'Pressure of Air Intake Manifold (kPa)',
    'EngIntakeManifold1Temp':'Temperature of Air Intake Manifold (C)',
    'SteeringWheelAngle1':'Steering Wheel Angle (radian)',
    'TotalVehicleDistanceHighRes':'Total Distance (m)',
    'TransmissionOilTemp':'Transmission Lube Temperature (C)',
    'TransOilLevel':'Transmission Lube Level (%)',
    'TransOilPress':'Transmission Lube Pressure (kPa)',
    'tse_TurboSpeed':'Turbo Speed (rpm)',
    'VehicleSpeed':'Vehicle Speed (km/hr)',
    'VehicleWeightVP15':'Vehicle Weight (kg)',
    'WheelBasedVehicleSpeed':'Vehicle Speed (Wheel-Based, km/hr)',
    'CPU load':'CPU Load (%)',
    'UTC hour':'UTC Hour',
    'UTC minute':'UTC Minute',
    'UTC second':'UTC Second',
    'UTC month':'UTC Month',
    'UTC day':'UTC Day',
    'UTC year':'UTC Year',
    'GPS altitude':'Altitude(m)',
    'GPS speed':'Speed (km/hr)' 
}

truck2dict = {
    'Time':'Time (s)',
    'C_1115_1_Alternator_250A':'1115 Alternator (A)',
    'C_1115_2_BattOut_100A':'1115 Battery Out (A)',
    'C_1115_3_Fridge_20A':'1115 Fridge (A)',
    'C_1115_4_Trailer_50A':'1115 Trailer (A)',
    'C_1115_5_Inventer_100A':'1115 Inverter (A)',
    'TotalVehicleDistanceHighRes':'Total Distance (m)',
    'EngineSpeed':'Engine Speed (rpm)',
    'WheelBasedVehicleSpeed':'Vehicle Speed (Wheel-Based, km/hr)',
    'asu_UreaTankTemp':'Urea Tank Temperature (C)',
    'EngineKeyPosition':'Key Position',
    'SteeringWheelAngle1':'Steering Wheel Angle',
    'BarometricPressure_BB1_X_E':'Outside Air Pressure (kPa)',
    'EngineAirIntakeTemp_BB1_X_E':'Temperature of Air Entering Vehicle (C)',
    'ActualEnginePercentTorque':'Engine Output Torque (%)',
    'ActualEnginePercentTorqueHiRes':'Engine Output Torque HiRes (%)',
    'DrvrDemandEngPercentTorque':'Driver Requested Torque (%)',
    'EngDemandPercentTorque':'Engine Requested Torque (%)',
    'EnginePercentLoadAtCurrentSpd':'Engine Load at Current Speed (%)',
    'EngineOilLevel':'Oil Level (%)',
    'EngineOilPressure':'Oil Pressure (kPa)',
    'EngineCoolantTemp_stat':'Coolant Temperature (C)',
    'EngineOilTemperature':'Oil Temperature (C)',
    'EngDslPrtclateFilterIntakePress':'Exhaust Back Pressure (kPa)',
    'EngExhaustGasTemp':'Temperature of Engine Byproducts (C)',
    'EngIntakeManifold1Press':'Pressure of Intake Manifold (kPa)',
    'EngIntakeManifold1Temp':'Temperature of Pre-Combustion Air (C)',
    'TransmissionOilTemp':'Transmission Lube Temperature (C)',
    'TransOilLevel':'Transmission Lube Level (%)',
    'TransOilPress':'Transmission Lube Pressure (kPa)',
    'RoadInclinationVP15':'Road Inclination (%)',
    'VehicleWeightVP15':'Vehicle Weight (kg)',
    'AmbientAirTemperature_V':'Outside Air Temperature (C)',
    'CruiseCtrlSetSpeed':'Cruise Control Speed (km/hr)',
    'VehicleSpeed':'Vehicle Speed (km/hr)',
    'BoostPressure':'Boost Pressure (kPa)',
    'tse_TurboSpeed':'Turbo Speed (rpm)',
    'CPU load':'CPU Load (%)',
    'UTC hour':'UTC Hour',
    'UTC minute':'UTC Minute',
    'UTC second':'UTC Second',
    'UTC month':'UTC Month',
    'UTC day':'UTC Day',
    'UTC year':'UTC Year',
    'GPS altitude':'Altitude (m)',
    'GPS speed':'Speed (km/hr)'
}

In [8]:
def readCsv(truck, records):
    return pd.read_csv(truck, nrows = records, header = [0])
    
def cullUtcCols(truckData):
    noUTC = truckData.drop(["UTC hour", "UTC minute", "UTC second", "UTC month", "UTC day", "UTC year"], axis = 1)
    return noUTC

def renameColumns(dataFrame, dictionary):
    """Takes a DataFrame and a conversion dictionary as parameters; uses the dictionary to rename all matching columns then returns the changed DataFrame."""
    return dataFrame.rename(columns = dictionary)

def removeUnnecessaryRows(dataFrame):
    """Finds threshold of Non-NA type using percentage and deletes rows."""
    length = len(dataFrame.columns)
    TValue = (rowNaNThresholdPercent / 100) * length
    print("Threshold value: " + str(int(TValue)))
    return dataFrame.dropna(thresh = int(TValue))

Cleaning of Truck 1 Data starts here

In [9]:
truck1data = readCsv(truck1csv, numberOfRecords)
truck1data = cullUtcCols(truck1data)
truck1data = removeUnnecessaryRows(truck1data)
truck1data = renameColumns(truck1data, truck1dict)

#Look at first 10 records in the DataFrame. Note: this does not work while the shape method below is uncommented.
truck1data.head(10)

# After cleaning, check the shape of the dataframe
#truck1data.shape

Threshold value: 33


,Time (s),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,Transmission Lube Temperature (C),Transmission Lube Level (%),Transmission Lube Pressure (kPa),Turbo Speed (rpm),Vehicle Speed (km/hr),Vehicle Weight (kg),"Vehicle Speed (Wheel-Based, km/hr)",CPU Load (%),Altitude(m),Speed (km/hr)
16,08/07/2019 07:54:04.559,0.006409,13.946441,97.500381,27.971695,0.277523,1.337072,0.145914,-0.033379,0.843061,...,27.84375,NaN,NaN,NaN,8.167969,18000.0,8.167969,40.0,0.0,6.1116
17,08/07/2019 07:54:04.659,0.007782,13.953765,96.985389,22.375448,0.275616,5.640116,0.134470,-0.004768,0.774395,...,27.84375,NaN,NaN,NaN,8.167969,18000.0,8.167969,40.0,0.0,6.1116
18,08/07/2019 07:54:04.759,0.007782,13.956970,98.330091,18.987945,0.273709,1.287480,0.126841,-0.014305,0.736248,...,27.84375,NaN,NaN,NaN,8.117188,18000.0,8.167969,40.0,0.0,6.1116
19,08/07/2019 07:54:04.859,0.007324,13.953307,94.076638,19.071870,0.277523,6.105516,0.136378,-0.071527,0.774395,...,27.84375,NaN,NaN,NaN,8.117188,18000.0,8.117188,88.0,0.0,6.6672
20,08/07/2019 07:54:04.959,0.007324,13.957885,96.365492,21.677348,0.264172,3.751812,0.128748,-0.071527,0.759136,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.0,6.6672
21,08/07/2019 07:54:05.059,0.005951,13.984894,104.071298,18.373770,0.277523,7.558938,0.126841,-0.023842,0.759136,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.0,6.6672
22,08/07/2019 07:54:05.159,0.007324,13.970703,96.727893,22.276265,0.269894,4.747463,0.132563,0.004768,0.881209,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.0,6.6672
23,08/07/2019 07:54:05.259,NaN,13.959258,90.080682,17.767224,0.271801,5.678264,0.130655,-0.004768,0.774395,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.0,6.6672
24,08/07/2019 07:54:05.359,NaN,13.956054,98.835546,25.068666,0.315671,5.674449,0.140192,0.004768,0.835431,...,27.84375,NaN,NaN,NaN,7.718750,18000.0,7.718750,88.0,0.0,6.6672
25,08/07/2019 07:54:05.459,NaN,13.961547,99.321927,22.695888,0.287060,5.663005,0.115396,-0.023842,0.751507,...,27.84375,NaN,NaN,NaN,7.718750,18000.0,7.718750,88.0,0.0,6.6672


Cleaning of Truck 2 Data starts here

In [3]:
truck2data = readCsv(truck2csv, numberOfRecords)
truck2data = cullUtcCols(truck2data)
truck2data = removeUnnecessaryRows(truck2data)
truck2data = renameColumns(truck2data, truck2dict)

#Look at first 10 records in the DataFrame. Note: this does not work while the shape method below is uncommented.
#truck2data.head(10)

# After cleaning, check the shape of the dataframe
#truck2data.shape

NameError: name 'readCsv' is not defined

In [60]:
truck1data.head(1000)

,Time (s),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,Transmission Lube Temperature (C),Transmission Lube Level (%),Transmission Lube Pressure (kPa),Turbo Speed (rpm),Vehicle Speed (km/hr),Vehicle Weight (kg),"Vehicle Speed (Wheel-Based, km/hr)",CPU Load (%),Altitude(m),Speed (km/hr)
16,08/07/2019 07:54:04.559,0.006409,13.946441,97.500381,27.971695,0.277523,1.337072,0.145914,-0.033379,0.843061,...,27.84375,NaN,NaN,NaN,8.167969,18000.0,8.167969,40.0,0.000000,6.1116
17,08/07/2019 07:54:04.659,0.007782,13.953765,96.985389,22.375448,0.275616,5.640116,0.134470,-0.004768,0.774395,...,27.84375,NaN,NaN,NaN,8.167969,18000.0,8.167969,40.0,0.000000,6.1116
18,08/07/2019 07:54:04.759,0.007782,13.956970,98.330091,18.987945,0.273709,1.287480,0.126841,-0.014305,0.736248,...,27.84375,NaN,NaN,NaN,8.117188,18000.0,8.167969,40.0,0.000000,6.1116
19,08/07/2019 07:54:04.859,0.007324,13.953307,94.076638,19.071870,0.277523,6.105516,0.136378,-0.071527,0.774395,...,27.84375,NaN,NaN,NaN,8.117188,18000.0,8.117188,88.0,0.000000,6.6672
20,08/07/2019 07:54:04.959,0.007324,13.957885,96.365492,21.677348,0.264172,3.751812,0.128748,-0.071527,0.759136,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.000000,6.6672
21,08/07/2019 07:54:05.059,0.005951,13.984894,104.071298,18.373770,0.277523,7.558938,0.126841,-0.023842,0.759136,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.000000,6.6672
22,08/07/2019 07:54:05.159,0.007324,13.970703,96.727893,22.276265,0.269894,4.747463,0.132563,0.004768,0.881209,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.000000,6.6672
23,08/07/2019 07:54:05.259,NaN,13.959258,90.080682,17.767224,0.271801,5.678264,0.130655,-0.004768,0.774395,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.000000,6.6672
24,08/07/2019 07:54:05.359,NaN,13.956054,98.835546,25.068666,0.315671,5.674449,0.140192,0.004768,0.835431,...,27.84375,NaN,NaN,NaN,7.718750,18000.0,7.718750,88.0,0.000000,6.6672
25,08/07/2019 07:54:05.459,NaN,13.961547,99.321927,22.695888,0.287060,5.663005,0.115396,-0.023842,0.751507,...,27.84375,NaN,NaN,NaN,7.718750,18000.0,7.718750,88.0,0.000000,6.6672


In [16]:
truck2data.head()

,Time (s),1115 Alternator (A),1115 Battery Out (A),1115 Fridge (A),1115 Trailer (A),1115 Inverter (A),Total Distance (m),Engine Speed (rpm),"Vehicle Speed (Wheel-Based, km/hr)",Urea Tank Temperature (C),...,Road Inclination (%),Vehicle Weight (kg),Outside Air Temperature (C),Cruise Control Speed (km/hr),Vehicle Speed (km/hr),Boost Pressure (kPa),Turbo Speed (rpm),CPU Load (%),Altitude (m),Speed (km/hr)
0,03/11/2019 14:49:09.038,-35.452125,4.949645,0.555047,-0.430114,-2.283131,NaN,1096.375,106.886719,NaN,...,2.4,18000.0,16.9375,NaN,106.886719,196.0,NaN,75.0,337.5,29.323334
1,03/11/2019 14:49:09.138,-35.480735,5.270085,0.626764,-0.420577,-2.294575,NaN,1098.875,107.101562,NaN,...,2.4,18000.0,16.9375,NaN,107.101562,198.0,NaN,75.0,337.5,29.323334
2,03/11/2019 14:49:09.238,-35.480735,5.197604,0.568780,-62.500000,-2.271687,NaN,1096.750,107.089844,NaN,...,2.4,18000.0,16.9375,NaN,107.089844,198.0,NaN,75.0,337.5,29.323334
3,03/11/2019 14:49:09.338,-35.452125,5.418860,0.638209,-62.500000,-2.290761,411617045.0,1098.375,106.921875,NaN,...,2.4,18000.0,16.9375,NaN,106.921875,200.0,NaN,75.0,337.5,29.323334
4,03/11/2019 14:49:09.438,-35.471199,5.228122,0.590143,-62.500000,-2.283131,411617045.0,1101.750,107.109375,NaN,...,2.4,18000.0,16.9375,NaN,107.109375,200.0,NaN,75.0,337.5,29.323334


In [17]:
truck1data

,Time (s),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,Transmission Lube Temperature (C),Transmission Lube Level (%),Transmission Lube Pressure (kPa),Turbo Speed (rpm),Vehicle Speed (km/hr),Vehicle Weight (kg),"Vehicle Speed (Wheel-Based, km/hr)",CPU Load (%),Altitude(m),Speed (km/hr)
16,08/07/2019 07:54:04.559,0.006409,13.946441,97.500381,27.971695,0.277523,1.337072,0.145914,-0.033379,0.843061,...,27.84375,NaN,NaN,NaN,8.167969,18000.0,8.167969,40.0,0.0,6.1116
17,08/07/2019 07:54:04.659,0.007782,13.953765,96.985389,22.375448,0.275616,5.640116,0.134470,-0.004768,0.774395,...,27.84375,NaN,NaN,NaN,8.167969,18000.0,8.167969,40.0,0.0,6.1116
18,08/07/2019 07:54:04.759,0.007782,13.956970,98.330091,18.987945,0.273709,1.287480,0.126841,-0.014305,0.736248,...,27.84375,NaN,NaN,NaN,8.117188,18000.0,8.167969,40.0,0.0,6.1116
19,08/07/2019 07:54:04.859,0.007324,13.953307,94.076638,19.071870,0.277523,6.105516,0.136378,-0.071527,0.774395,...,27.84375,NaN,NaN,NaN,8.117188,18000.0,8.117188,88.0,0.0,6.6672
20,08/07/2019 07:54:04.959,0.007324,13.957885,96.365492,21.677348,0.264172,3.751812,0.128748,-0.071527,0.759136,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.0,6.6672
21,08/07/2019 07:54:05.059,0.005951,13.984894,104.071298,18.373770,0.277523,7.558938,0.126841,-0.023842,0.759136,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.0,6.6672
22,08/07/2019 07:54:05.159,0.007324,13.970703,96.727893,22.276265,0.269894,4.747463,0.132563,0.004768,0.881209,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.0,6.6672
23,08/07/2019 07:54:05.259,NaN,13.959258,90.080682,17.767224,0.271801,5.678264,0.130655,-0.004768,0.774395,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.0,6.6672
24,08/07/2019 07:54:05.359,NaN,13.956054,98.835546,25.068666,0.315671,5.674449,0.140192,0.004768,0.835431,...,27.84375,NaN,NaN,NaN,7.718750,18000.0,7.718750,88.0,0.0,6.6672
25,08/07/2019 07:54:05.459,NaN,13.961547,99.321927,22.695888,0.287060,5.663005,0.115396,-0.023842,0.751507,...,27.84375,NaN,NaN,NaN,7.718750,18000.0,7.718750,88.0,0.0,6.6672


In [18]:
truck2data

,Time (s),1115 Alternator (A),1115 Battery Out (A),1115 Fridge (A),1115 Trailer (A),1115 Inverter (A),Total Distance (m),Engine Speed (rpm),"Vehicle Speed (Wheel-Based, km/hr)",Urea Tank Temperature (C),...,Road Inclination (%),Vehicle Weight (kg),Outside Air Temperature (C),Cruise Control Speed (km/hr),Vehicle Speed (km/hr),Boost Pressure (kPa),Turbo Speed (rpm),CPU Load (%),Altitude (m),Speed (km/hr)
0,03/11/2019 14:49:09.038,-35.452125,4.949645,0.555047,-0.430114,-2.283131,NaN,1096.375,106.886719,NaN,...,2.4,18000.0,16.9375,NaN,106.886719,196.0,NaN,75.0,337.500000,29.323334
1,03/11/2019 14:49:09.138,-35.480735,5.270085,0.626764,-0.420577,-2.294575,NaN,1098.875,107.101562,NaN,...,2.4,18000.0,16.9375,NaN,107.101562,198.0,NaN,75.0,337.500000,29.323334
2,03/11/2019 14:49:09.238,-35.480735,5.197604,0.568780,-62.500000,-2.271687,NaN,1096.750,107.089844,NaN,...,2.4,18000.0,16.9375,NaN,107.089844,198.0,NaN,75.0,337.500000,29.323334
3,03/11/2019 14:49:09.338,-35.452125,5.418860,0.638209,-62.500000,-2.290761,411617045.0,1098.375,106.921875,NaN,...,2.4,18000.0,16.9375,NaN,106.921875,200.0,NaN,75.0,337.500000,29.323334
4,03/11/2019 14:49:09.438,-35.471199,5.228122,0.590143,-62.500000,-2.283131,411617045.0,1101.750,107.109375,NaN,...,2.4,18000.0,16.9375,NaN,107.109375,200.0,NaN,75.0,337.500000,29.323334
5,03/11/2019 14:49:09.538,-35.452125,5.544747,0.529870,-62.500000,-2.283131,411617045.0,1096.875,107.238281,NaN,...,2.4,18000.0,16.9375,NaN,107.238281,200.0,NaN,75.0,337.500000,29.323334
6,03/11/2019 14:49:09.638,-35.442588,5.285344,0.580987,-62.500000,-2.256428,411617045.0,1102.750,107.300781,NaN,...,2.4,18000.0,16.9375,NaN,107.300781,202.0,NaN,75.0,337.500000,29.323334
7,03/11/2019 14:49:09.738,-35.461662,5.151827,0.558862,-62.500000,-2.264057,411617045.0,1103.000,107.171875,NaN,...,2.4,18000.0,16.9375,NaN,107.171875,202.0,NaN,75.0,337.500000,29.323334
8,03/11/2019 14:49:09.838,-35.471199,5.010681,0.548180,-62.500000,-2.241169,411617045.0,1102.125,107.316406,NaN,...,2.4,18000.0,16.9375,NaN,107.316406,202.0,NaN,75.0,337.500000,29.323334
9,03/11/2019 14:49:09.938,-35.461662,5.350195,0.574121,-62.500000,-2.248798,411617045.0,1105.125,107.316406,NaN,...,2.4,18000.0,16.9375,NaN,107.316406,202.0,NaN,75.0,337.500000,29.323334


In [19]:
print(truck1data.describe())

       1730 Automatic Start & Stop (V)  1730 Batteries (V)  \
count                        17.000000           84.000000   
mean                          0.003312           13.914517   
std                           0.003327            0.059048   
min                           0.000458           13.738155   
25%                           0.000458           13.844816   
50%                           0.000916           13.953307   
75%                           0.007324           13.963378   
max                           0.007782           13.984894   

       4649 Alternator (A)  4649 Battery Out (A)  4649 Trailer (A)  \
count            84.000000             84.000000         84.000000   
mean            141.373703             21.071029          0.278205   
std              51.281053              2.581490          0.057946   
min              89.632448             17.404822          0.149729   
25%              95.113775             18.790532          0.267510   
50%             100.0

In [20]:
print(truck2data.describe())

       1115 Alternator (A)  1115 Battery Out (A)  1115 Fridge (A)  \
count           100.000000            100.000000       100.000000   
mean            -35.470150              5.380293         0.560174   
std               0.011731              0.455941         0.037670   
min             -35.499809              4.709316         0.480278   
25%             -35.480735              5.086023         0.532158   
50%             -35.471199              5.279622         0.557336   
75%             -35.461662              5.486572         0.585374   
max             -35.442588              7.524605         0.657282   

       1115 Trailer (A)  1115 Inverter (A)  Total Distance (m)  \
count        100.000000         100.000000        9.700000e+01   
mean         -60.016899          -2.239872        4.116172e+08   
std           12.225942           0.033394        8.374057e+01   
min          -62.500000          -2.332723        4.116170e+08   
25%          -62.500000          -2.260243      

In [21]:
df = pd.concat([truck1data, truck2data], sort=False)

In [22]:
df.head()

,Time (s),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,1115 Fridge (A),1115 Trailer (A),1115 Inverter (A),Steering Wheel Angle,Engine Output Torque HiRes (%),Coolant Temperature (C),Pressure of Intake Manifold (kPa),Temperature of Pre-Combustion Air (C),Road Inclination (%),Altitude (m)
16,08/07/2019 07:54:04.559,0.006409,13.946441,97.500381,27.971695,0.277523,1.337072,0.145914,-0.033379,0.843061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,08/07/2019 07:54:04.659,0.007782,13.953765,96.985389,22.375448,0.275616,5.640116,0.134470,-0.004768,0.774395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,08/07/2019 07:54:04.759,0.007782,13.956970,98.330091,18.987945,0.273709,1.287480,0.126841,-0.014305,0.736248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,08/07/2019 07:54:04.859,0.007324,13.953307,94.076638,19.071870,0.277523,6.105516,0.136378,-0.071527,0.774395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,08/07/2019 07:54:04.959,0.007324,13.957885,96.365492,21.677348,0.264172,3.751812,0.128748,-0.071527,0.759136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.describe()

,1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),4649 Battery Voltage (V),...,1115 Fridge (A),1115 Trailer (A),1115 Inverter (A),Steering Wheel Angle,Engine Output Torque HiRes (%),Coolant Temperature (C),Pressure of Intake Manifold (kPa),Temperature of Pre-Combustion Air (C),Road Inclination (%),Altitude (m)
count,17.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,...,100.000000,100.000000,100.000000,100.000000,100.000,100.0,100.000000,100.0,100.000000,100.000000
mean,0.003312,13.914517,141.373703,21.071029,0.278205,5.071672,0.079906,0.054042,-1.293202,12.349295,...,0.560174,-60.016899,-2.239872,-0.268531,1.875,79.0,100.100000,33.0,2.612000,339.678001
std,0.003327,0.059048,51.281053,2.581490,0.057946,2.110573,0.039942,0.087493,2.758513,0.198262,...,0.037670,12.225942,0.033394,0.041072,0.000,0.0,7.260659,0.0,0.255556,1.563701
min,0.000458,13.738155,89.632448,17.404822,0.149729,0.211719,0.016213,-0.071527,-17.177844,12.172580,...,0.480278,-62.500000,-2.332723,-0.357398,1.875,79.0,82.000000,33.0,2.200000,337.500000
25%,0.000458,13.844816,95.113775,18.790532,0.267510,3.794728,0.042439,-0.023842,-3.295949,12.173953,...,0.532158,-62.500000,-2.260243,-0.298316,1.875,79.0,97.000000,33.0,2.400000,338.500000
50%,0.000916,13.953307,100.027657,21.122301,0.281338,5.481804,0.086786,0.014305,0.713359,12.174868,...,0.557336,-62.500000,-2.241169,-0.275367,1.875,79.0,101.000000,33.0,2.600000,339.600006
75%,0.007324,13.963378,194.552529,22.693027,0.321393,6.383040,0.117781,0.147822,0.774395,12.545663,...,0.585374,-62.500000,-2.225910,-0.235816,1.875,79.0,106.000000,33.0,2.800000,340.700012
max,0.007782,13.984894,210.502976,27.971695,0.382429,9.744793,0.145914,0.205043,0.881209,12.661479,...,0.657282,-0.416762,-2.119097,-0.175758,1.875,79.0,110.000000,33.0,3.200000,343.299988


In [24]:
print(df.describe())

       1730 Automatic Start & Stop (V)  1730 Batteries (V)  \
count                        17.000000           84.000000   
mean                          0.003312           13.914517   
std                           0.003327            0.059048   
min                           0.000458           13.738155   
25%                           0.000458           13.844816   
50%                           0.000916           13.953307   
75%                           0.007324           13.963378   
max                           0.007782           13.984894   

       4649 Alternator (A)  4649 Battery Out (A)  4649 Trailer (A)  \
count            84.000000             84.000000         84.000000   
mean            141.373703             21.071029          0.278205   
std              51.281053              2.581490          0.057946   
min              89.632448             17.404822          0.149729   
25%              95.113775             18.790532          0.267510   
50%             100.0

In [25]:
print(truck1data.groupby('Vehicle Weight (kg)'))

In [26]:
print(truck1data)

                   Time (s)  1730 Automatic Start & Stop (V)  \
16  08/07/2019 07:54:04.559                         0.006409   
17  08/07/2019 07:54:04.659                         0.007782   
18  08/07/2019 07:54:04.759                         0.007782   
19  08/07/2019 07:54:04.859                         0.007324   
20  08/07/2019 07:54:04.959                         0.007324   
21  08/07/2019 07:54:05.059                         0.005951   
22  08/07/2019 07:54:05.159                         0.007324   
23  08/07/2019 07:54:05.259                              NaN   
24  08/07/2019 07:54:05.359                              NaN   
25  08/07/2019 07:54:05.459                              NaN   
26  08/07/2019 07:54:05.559                              NaN   
27  08/07/2019 07:54:05.659                              NaN   
28  08/07/2019 07:54:05.759                              NaN   
29  08/07/2019 07:54:05.859                              NaN   
30  08/07/2019 07:54:05.959             

In [56]:
byWeight = truck1data.groupby('Vehicle Weight (kg)')

In [57]:
byWeight.head()

,Time (s),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,Transmission Lube Temperature (C),Transmission Lube Level (%),Transmission Lube Pressure (kPa),Turbo Speed (rpm),Vehicle Speed (km/hr),Vehicle Weight (kg),"Vehicle Speed (Wheel-Based, km/hr)",CPU Load (%),Altitude(m),Speed (km/hr)
16,08/07/2019 07:54:04.559,0.006409,13.946441,97.500381,27.971695,0.277523,1.337072,0.145914,-0.033379,0.843061,...,27.84375,NaN,NaN,NaN,8.167969,18000.0,8.167969,40.0,0.0,6.1116
17,08/07/2019 07:54:04.659,0.007782,13.953765,96.985389,22.375448,0.275616,5.640116,0.134470,-0.004768,0.774395,...,27.84375,NaN,NaN,NaN,8.167969,18000.0,8.167969,40.0,0.0,6.1116
18,08/07/2019 07:54:04.759,0.007782,13.956970,98.330091,18.987945,0.273709,1.287480,0.126841,-0.014305,0.736248,...,27.84375,NaN,NaN,NaN,8.117188,18000.0,8.167969,40.0,0.0,6.1116
19,08/07/2019 07:54:04.859,0.007324,13.953307,94.076638,19.071870,0.277523,6.105516,0.136378,-0.071527,0.774395,...,27.84375,NaN,NaN,NaN,8.117188,18000.0,8.117188,88.0,0.0,6.6672
20,08/07/2019 07:54:04.959,0.007324,13.957885,96.365492,21.677348,0.264172,3.751812,0.128748,-0.071527,0.759136,...,27.84375,NaN,NaN,NaN,7.820312,18000.0,7.820312,88.0,0.0,6.6672


In [58]:
print(byWeight.describe())

                    1730 Automatic Start & Stop (V)                      \
                                              count      mean       std   
Vehicle Weight (kg)                                                       
18000.0                                        17.0  0.003312  0.003327   

                                                                       \
                          min       25%       50%       75%       max   
Vehicle Weight (kg)                                                     
18000.0              0.000458  0.000458  0.000916  0.007324  0.007782   

                    1730 Batteries (V)             ... Altitude(m)       \
                                 count       mean  ...         75%  max   
Vehicle Weight (kg)                                ...                    
18000.0                           84.0  13.914517  ...         0.0  0.0   

                    Speed (km/hr)                                              \
                        

In [63]:
byWeight2 = truck2data.groupby('Vehicle Weight (kg)', axis=0)

In [64]:
print(byWeight2.describe())

                    1115 Alternator (A)                                 \
                                  count      mean       std        min   
Vehicle Weight (kg)                                                      
18000.0                           100.0 -35.47015  0.011731 -35.499809   

                                                                 \
                           25%        50%        75%        max   
Vehicle Weight (kg)                                               
18000.0             -35.480735 -35.471199 -35.461662 -35.442588   

                    1115 Battery Out (A)            ... Altitude (m)  \
                                   count      mean  ...          75%   
Vehicle Weight (kg)                                 ...                
18000.0                            100.0  5.380293  ...   340.700012   

                                Speed (km/hr)                                  \
                            max         count       mean       s

In [65]:
byWeight2

In [70]:
print(byWeight2.describe())

                    1115 Alternator (A)                                 \
                                  count      mean       std        min   
Vehicle Weight (kg)                                                      
18000.0                           100.0 -35.47015  0.011731 -35.499809   

                                                                 \
                           25%        50%        75%        max   
Vehicle Weight (kg)                                               
18000.0             -35.480735 -35.471199 -35.461662 -35.442588   

                    1115 Battery Out (A)            ... Altitude (m)  \
                                   count      mean  ...          75%   
Vehicle Weight (kg)                                 ...                
18000.0                            100.0  5.380293  ...   340.700012   

                                Speed (km/hr)                                  \
                            max         count       mean       s

In [68]:
byWeight2 = truck2data.groupby('Vehicle Weight (kg)')

for key, item in byWeight2:
    print(byWeight2.get_group(key), "\n\n")

                   Time (s)  1115 Alternator (A)  1115 Battery Out (A)  \
0   03/11/2019 14:49:09.038           -35.452125              4.949645   
1   03/11/2019 14:49:09.138           -35.480735              5.270085   
2   03/11/2019 14:49:09.238           -35.480735              5.197604   
3   03/11/2019 14:49:09.338           -35.452125              5.418860   
4   03/11/2019 14:49:09.438           -35.471199              5.228122   
5   03/11/2019 14:49:09.538           -35.452125              5.544747   
6   03/11/2019 14:49:09.638           -35.442588              5.285344   
7   03/11/2019 14:49:09.738           -35.461662              5.151827   
8   03/11/2019 14:49:09.838           -35.471199              5.010681   
9   03/11/2019 14:49:09.938           -35.461662              5.350195   
10  03/11/2019 14:49:10.038           -35.442588              4.873350   
11  03/11/2019 14:49:10.138           -35.480735              4.991608   
12  03/11/2019 14:49:10.238           

In [28]:
df = truck1data

This sums all the data around the grouping by 'Vehicle Weight'

In [29]:
df.groupby(['Vehicle Weight (kg)'], as_index=False).sum()

,Vehicle Weight (kg),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,Total Distance (m),Transmission Lube Temperature (C),Transmission Lube Level (%),Transmission Lube Pressure (kPa),Turbo Speed (rpm),Vehicle Speed (km/hr),"Vehicle Speed (Wheel-Based, km/hr)",CPU Load (%),Altitude(m),Speed (km/hr)
0,18000.0,0.056306,1168.819409,11875.391012,1769.96643,23.369192,426.020447,6.712062,4.539559,-108.628977,...,2.066785e+10,2343.5625,0.0,0.0,0.0,774.949219,773.996094,3972.0,0.0,743.392783


In [30]:
df2 = truck2data

This groups by the 'Vehicle Weight' then uses the mean as the new values in the rest of the columns

In [31]:
df2.groupby(['Vehicle Weight (kg)'], as_index=False).mean()

,Vehicle Weight (kg),1115 Alternator (A),1115 Battery Out (A),1115 Fridge (A),1115 Trailer (A),1115 Inverter (A),Total Distance (m),Engine Speed (rpm),"Vehicle Speed (Wheel-Based, km/hr)",Urea Tank Temperature (C),...,Transmission Lube Pressure (kPa),Road Inclination (%),Outside Air Temperature (C),Cruise Control Speed (km/hr),Vehicle Speed (km/hr),Boost Pressure (kPa),Turbo Speed (rpm),CPU Load (%),Altitude (m),Speed (km/hr)
0,18000.0,-35.47015,5.380293,0.560174,-60.016899,-2.239872,4.116172e+08,1108.89375,107.984023,17.0,...,NaN,2.612,16.9025,NaN,107.986406,198.36,70997.471354,62.88,339.678001,29.762155


In [32]:
df.groupby(['Vehicle Weight (kg)'], as_index=False).mean()

,Vehicle Weight (kg),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,Total Distance (m),Transmission Lube Temperature (C),Transmission Lube Level (%),Transmission Lube Pressure (kPa),Turbo Speed (rpm),Vehicle Speed (km/hr),"Vehicle Speed (Wheel-Based, km/hr)",CPU Load (%),Altitude(m),Speed (km/hr)
0,18000.0,0.003312,13.914517,141.373703,21.071029,0.278205,5.071672,0.079906,0.054042,-1.293202,...,2.460459e+08,27.899554,NaN,NaN,NaN,9.225586,9.214239,47.285714,0.0,8.849914


In [33]:
dftest = truck1data

In [34]:
dftest.groupby(['Vehicle Weight (kg)'], as_index=False).mean()

,Vehicle Weight (kg),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,Total Distance (m),Transmission Lube Temperature (C),Transmission Lube Level (%),Transmission Lube Pressure (kPa),Turbo Speed (rpm),Vehicle Speed (km/hr),"Vehicle Speed (Wheel-Based, km/hr)",CPU Load (%),Altitude(m),Speed (km/hr)
0,18000.0,0.003312,13.914517,141.373703,21.071029,0.278205,5.071672,0.079906,0.054042,-1.293202,...,2.460459e+08,27.899554,NaN,NaN,NaN,9.225586,9.214239,47.285714,0.0,8.849914


In [35]:
plt.plot(truck1data[1])

KeyError: 1

In [14]:
#Bill , convert timestamp to datetime for identifying Vehicle type
import datetime

truckonetype =  pd.to_datetime(truck1data['Time (s)'])

truckonetype.head(1000)

16    2019-08-07 07:54:04.559
17    2019-08-07 07:54:04.659
18    2019-08-07 07:54:04.759
19    2019-08-07 07:54:04.859
20    2019-08-07 07:54:04.959
21    2019-08-07 07:54:05.059
22    2019-08-07 07:54:05.159
23    2019-08-07 07:54:05.259
24    2019-08-07 07:54:05.359
25    2019-08-07 07:54:05.459
26    2019-08-07 07:54:05.559
27    2019-08-07 07:54:05.659
28    2019-08-07 07:54:05.759
29    2019-08-07 07:54:05.859
30    2019-08-07 07:54:05.959
31    2019-08-07 07:54:06.059
32    2019-08-07 07:54:06.159
33    2019-08-07 07:54:06.259
34    2019-08-07 07:54:06.359
35    2019-08-07 07:54:06.459
36    2019-08-07 07:54:06.559
37    2019-08-07 07:54:06.659
38    2019-08-07 07:54:06.759
39    2019-08-07 07:54:06.859
40    2019-08-07 07:54:06.959
41    2019-08-07 07:54:07.059
42    2019-08-07 07:54:07.159
43    2019-08-07 07:54:07.259
44    2019-08-07 07:54:07.359
45    2019-08-07 07:54:07.459
                ...          
956   2019-08-05 10:24:26.742
957   2019-08-05 10:24:26.842
958   2019

In [64]:

truckTwotype =  pd.to_datetime(truck2data['Time (s)'])

truckTwotype.head(1000)

0     2019-03-11 14:49:09.038
1     2019-03-11 14:49:09.138
2     2019-03-11 14:49:09.238
3     2019-03-11 14:49:09.338
4     2019-03-11 14:49:09.438
5     2019-03-11 14:49:09.538
6     2019-03-11 14:49:09.638
7     2019-03-11 14:49:09.738
8     2019-03-11 14:49:09.838
9     2019-03-11 14:49:09.938
10    2019-03-11 14:49:10.038
11    2019-03-11 14:49:10.138
12    2019-03-11 14:49:10.238
13    2019-03-11 14:49:10.338
14    2019-03-11 14:49:10.438
15    2019-03-11 14:49:10.538
16    2019-03-11 14:49:10.638
17    2019-03-11 14:49:10.738
18    2019-03-11 14:49:10.838
19    2019-03-11 14:49:10.938
20    2019-03-11 14:49:11.038
21    2019-03-11 14:49:11.138
22    2019-03-11 14:49:11.238
23    2019-03-11 14:49:11.338
24    2019-03-11 14:49:11.438
25    2019-03-11 14:49:11.538
26    2019-03-11 14:49:11.638
27    2019-03-11 14:49:11.738
28    2019-03-11 14:49:11.838
29    2019-03-11 14:49:11.938
                ...          
970   2019-03-11 14:50:46.038
971   2019-03-11 14:50:46.138
972   2019